# Qwen3

## 模型推理

### 使用transformers
在 transformers 中使用 Qwen3-8B ：

In [ ]:
# 已安装，Version: 4.52.4
!pip install transformers -U

In [1]:
# 指定GPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"  # 0-based编号，GPU7是第8块卡

# 屏蔽警告信息
import warnings
warnings.filterwarnings(
    "ignore",
    message=r"You are using `torch\.load` with `weights_only=False`.*",
    category=FutureWarning,
    module=r"torch\.serialization",
)

In [2]:
# 使用下载好的模型：/data/joey.wang/llm/models/pretrained/llm/Qwen3-8B

In [3]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "/data/joey.wang/llm/models/pretrained/llm/Qwen3-8B"
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True  # Switch between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0
thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  1.78it/s]


thinking content: <think>
Okay, the user wants a short introduction to large language models. Let me start by defining what they are. They're AI systems trained on vast amounts of text data, right? So I should mention their training data size and the purpose, like understanding and generating human-like text.

Wait, maybe I should explain the key features. They can handle multiple languages, answer questions, write stories, etc. Oh, and they're called "large" because of their scale—parameters, data, and computational resources. That's important to highlight.

I should also touch on their applications. Like, they're used in chatbots, customer service, content creation. Maybe mention some examples like GPT or BERT? But the user might not need specific names, just a general idea.

Also, the user might be looking for a concise overview, so I need to keep it brief. Avoid jargon but still be accurate. Make sure to mention the training process, maybe something about neural networks. But keep 

In [4]:
print(text)

<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant



禁用思考模式，只需对参数 enable_thinking 进行如下修改：

In [5]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0
thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: A large language model (LLM) is an advanced type of artificial intelligence that is trained on vast amounts of text data to understand and generate human-like text. These models can perform a wide range of tasks, such as answering questions, writing stories, coding, and translating languages. LLMs are characterized by their massive size—often containing billions of parameters—which allows them to capture complex patterns in language and provide more accurate and contextually relevant responses. They are widely used in various applications, including chatbots, virtual assistants, content creation, and data analysis.


查看 enable_thinking 设置为False 做了什么？

In [6]:
print(text)

<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant
<think>

</think>




使用 /think 和 /no_think 控制模型的思考模式：

In [7]:
# prepare the model input
prompt = "Give me a short introduction to large language model. /no_think"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0
thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>

</think>
content: A large language model (LLM) is an advanced type of artificial intelligence that is trained on vast amounts of text data to understand and generate human-like text. These models can perform a wide range of tasks, such as answering questions, writing stories, coding, translating languages, and more. LLMs are built using deep learning techniques, particularly transformer architectures, which allow them to process and generate text efficiently. Their ability to comprehend context and produce coherent responses makes them powerful tools in various applications, from customer service to content creation.


### 使用魔搭 API-Inference
使用魔搭免费提供的 Qwen3 API-Inference推理：

In [8]:
# MODELSCOPE_SDK_TOKEN = os.environ.get('MODELSCOPE_SDK_TOKEN')
MODELSCOPE_SDK_TOKEN='ms-c5c215e6-c2c6-444b-8305-469cc21e243b'
from openai import OpenAI
client = OpenAI(
    base_url='https://api-inference.modelscope.cn/v1/',
    api_key=MODELSCOPE_SDK_TOKEN, # ModelScope Token
)

# set extra_body for thinking control
extra_body = {
    # enable thinking, set to False to disable
    "enable_thinking": True,
    # use thinking_budget to contorl num of tokens used for thinking
    # "thinking_budget": 4096
}
response = client.chat.completions.create(
    model='Qwen/Qwen3-32B',  # ModelScope Model-Id
    messages=[
        {
          'role': 'user',
          'content': '9.9和9.11谁大'
        }
    ],
    stream=True,
    extra_body=extra_body
)
done_thinking = False
for chunk in response:
    thinking_chunk = chunk.choices[0].delta.reasoning_content
    answer_chunk = chunk.choices[0].delta.content
    if thinking_chunk != '':
        print(thinking_chunk, end='', flush=True)
    elif answer_chunk != '':
        if not done_thinking:
            print('\n\n === Final Answer ===\n')
            done_thinking = True
        print(answer_chunk, end='', flush=True)

嗯，用户问的是9.9和9.11哪个更大。首先，我需要确认这两个数字的结构。看起来都是小数，但可能用户对小数点后的位数有疑问。比如，9.9可能被看作是9.90，而9.11则是两位小数。这时候比较的话，应该先统一小数位数，再逐位比较。

首先，比较整数部分，两者都是9，所以相同。接下来比较小数部分。9.9的小数部分是0.9，而9.11的小数部分是0.11。这时候可能需要将它们转换成相同的小数位数来比较。比如，把9.9写成9.90，这样小数部分就是90和11。显然，90比11大，所以9.90大于9.11。因此，9.9比9.11大。

不过，也有可能用户对小数的理解有误，比如认为9.11是两位小数，而9.9是一位小数，可能误以为9.11更大。这时候需要解释小数比较的方法，即先看整数部分，再逐位比较小数部分，从左到右，直到分出大小为止。

另外，可能用户是刚开始学习小数比较，容易混淆位数和数值的大小。这时候需要举例子说明，比如0.9和0.11，其实0.9更大，因为第一位小数9比1大，后面的数字不需要再比较了。同样的道理，9.9和9.11比较，小数部分第一位9和1，9更大，所以整体9.9更大。

再检查一下是否有其他可能性，比如是否用户输入有误，或者有没有考虑单位的问题。但题目中没有提到单位，应该只是纯数字比较。所以结论应该是9.9大于9.11。

 === Final Answer ===

9.9 和 9.11 的比较可以通过以下步骤进行：

1. **统一小数位数**：  
   将 9.9 补零为 **9.90**，以便与 9.11 对齐小数位数。

2. **逐位比较**：  
   - **整数部分**：两者均为 9，相等。  
   - **小数部分**：  
     - 第一位小数：9.90 的 **9** 与 9.11 的 **1** 比较，**9 > 1**。  
     - 后续位数无需再比较，直接得出结论。

**结论**：  
**9.9 > 9.11**。  
（因为小数部分 0.90 明显大于 0.11）

### 使用ms-swift

In [ ]:
!pip install vllm -U
!pip install git+https://github.com/modelscope/ms-swift.git

思考模式推理：

In [9]:
nproc_per_node=1
CUDA_VISIBLE_DEVICES=5 NPROC_PER_NODE=$nproc_per_node swift infer \
    --model /data/joey.wang/llm/models/pretrained/llm/Qwen3-8B \
    --infer_backend vllm \
    --max_model_len 2048 \
    --max_new_tokens 1024 \
    --stream true

SyntaxError: invalid decimal literal (562069236.py, line 3)

非思考模式界面推理：

In [ ]:
CUDA_VISIBLE_DEVICES=0 \
swift app \
    --model Qwen/Qwen3-8B \
    --infer_backend vllm \
    --stream true \
    --max_new_tokens 1024 \
    --max_model_len 2048 \
    --response_prefix '<think>\n\n</think>\n\n'


## 模型微调

非常高兴听到Qwen3和Qwen3-MoE的开源， [ms-swift](https://github.com/modelscope/ms-swift)大模型训练框架首发支持了Qwen3/Qwen3-MoE的**CPT/SFT/DPO/GRPO**，同时支持了Qwen3/Qwen3-MoE的**Megatron**训练(CPT/SFT)实现，在MoE模型上相比transformers实现的训练速度**快10倍**。

我们将展示可运行的微调demo，并给出自定义数据集的格式。

在开始微调之前，请确保您的环境已准备妥当。

In [ ]:
!pip install git+https://github.com/modelscope/ms-swift.git

!pip install liger-kernel transformers -U

### Qwen3-8B SFT

对Qwen3-8B进行训练的脚本如下，可在魔搭提供的**免费算力A10**中运行：https://modelscope.cn/my/mynotebook


In [ ]:
# 训练显存：22GB
# 你可以指定`--dataset AI-ModelScope/alpaca-gpt4-data-zh`来跑通实验
!CUDA_VISIBLE_DEVICES=0 \
swift sft \
    --model Qwen/Qwen3-8B \
    --train_type lora \
    --dataset '<dataset-path>' \
    --torch_dtype bfloat16 \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --learning_rate 1e-4 \
    --lora_rank 8 \
    --lora_alpha 32 \
    --target_modules all-linear \
    --gradient_accumulation_steps 4 \
    --eval_steps 50 \
    --save_steps 50 \
    --save_total_limit 2 \
    --logging_steps 5 \
    --max_length 2048 \
    --output_dir output \
    --warmup_ratio 0.05 \
    --dataloader_num_workers 4 \
    --packing true \
    --use_liger_kernel true \
    --attn_impl flash_attn

**自定义数据集**格式如下（system字段可选），指定`--dataset <dataset_path>`即可：

参考自定义数据集文档：https://swift.readthedocs.io/zh-cn/latest/Customization/%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E9%9B%86.html

```jsonl
{"messages": [{"role": "user", "content": "浙江的省会在哪？"}, {"role": "assistant", "content": "<think>\nxxx\n</think>\n\n浙江的省会在杭州。"}]}
{"messages": [{"role": "user", "content": "浙江的省会在哪？ /no_think"}, {"role": "assistant", "content": "<think>\n\n</think>\n\n浙江的省会在杭州。"}]}
```

**10分钟快速自我认知微调Demo**（显存占用：22GB）

In [ ]:
!CUDA_VISIBLE_DEVICES=7 \
swift sft \
    --model Qwen/Qwen3-8B \
    --train_type lora \
    --dataset 'swift/Qwen3-SFT-Mixin#2000' \
              'swift/self-cognition:qwen3#600' \
    --torch_dtype bfloat16 \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --learning_rate 1e-4 \
    --lora_rank 8 \
    --lora_alpha 32 \
    --target_modules all-linear \
    --gradient_accumulation_steps 16 \
    --eval_steps 50 \
    --save_steps 50 \
    --save_total_limit 2 \
    --logging_steps 5 \
    --max_length 2048 \
    --output_dir output \
    --warmup_ratio 0.05 \
    --dataloader_num_workers 4 \
    --use_liger_kernel true \
    --model_author swift \
    --model_name swift-robot

run sh: `/usr/local/bin/python /usr/local/lib/python3.11/site-packages/swift/cli/sft.py --model Qwen/Qwen3-8B --train_type lora --dataset swift/Qwen3-SFT-Mixin#2000 swift/self-cognition:qwen3#600 --torch_dtype bfloat16 --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --learning_rate 1e-4 --lora_rank 8 --lora_alpha 32 --target_modules all-linear --gradient_accumulation_steps 16 --eval_steps 50 --save_steps 50 --save_total_limit 2 --logging_steps 5 --max_length 2048 --output_dir output --warmup_ratio 0.05 --dataloader_num_workers 4 --use_liger_kernel true --model_author swift --model_name swift-robot`
[2025-04-30 06:59:52,362] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[INFO:swift] Successfully registered `/usr/local/lib/python3.11/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift.hub.hub] Downloading the model 


显存占用：22GiB


推理测试微调效果：

In [ ]:
CUDA_VISIBLE_DEVICES=0 \
swift app \
    --adapters output/vx-xxx/checkpoint-xxx \
    --stream true \
    --temperature 0 \
    --max_new_tokens 2048

使用Python的方式推理：（transformers or swift）

In [ ]:
from modelscope import snapshot_download

# model_dir = snapshot_download('Qwen/Qwen3-8B')
model_dir = '/data/joey.wang/llm/models/pretrained/llm/Qwen3-8B'
adapter_dir = 'output/vx-xxx/checkpoint-xxx'


def infer_hf():
    from transformers import AutoModelForCausalLM, AutoTokenizer
    from peft import PeftModel

    model = AutoModelForCausalLM.from_pretrained(
        model_dir, torch_dtype='auto', device_map='auto', trust_remote_code=True)
    model = PeftModel.from_pretrained(model, adapter_dir)

    tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

    messages = [{
        'role': 'user',
        'content': 'who are you?'
    }]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors='pt', add_special_tokens=False).to(model.device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=512, do_sample=False)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(f'response: {response}')
    return response


def infer_swift():
    from swift.llm import InferRequest, RequestConfig, PtEngine

    engine = PtEngine(model_dir, adapters=[adapter_dir])

    messages = [{
        'role': 'user',
        'content': 'who are you?'
    }]
    request_config = RequestConfig(max_tokens=512, temperature=0)
    resp_list = engine.infer([InferRequest(messages=messages)], request_config=request_config)
    response = resp_list[0].choices[0].message.content
    print(f'response: {response}')
    return response

In [3]:
response = infer_hf()

2025-04-30 08:11:56.755620: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-30 08:11:56.795927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-30 08:11:57.940359: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2025-04-30 08:11:59,178] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


response: <think>
Okay, the user asked, "who are you?" I need to respond appropriately. First, I should introduce myself as an AI assistant developed by swift. I should mention my name, swift-robot, and my purpose to assist with information and tasks. I should keep the response friendly and open-ended, inviting them to ask questions. I should avoid any technical jargon and make sure the tone is approachable. Let me check if there's anything else I need to include. Maybe a brief mention of my capabilities, like answering questions, providing information, and helping with tasks. Also, I should make sure to use proper grammar and punctuation. Alright, that should cover it.
</think>

Hello! I am swift-robot, an artificial intelligence assistant developed by swift. My purpose is to assist you in answering questions, providing information, and helping with various tasks. If you have any questions or need assistance, feel free to ask me!


In [2]:
response2 = infer_swift()

2025-04-30 08:13:33.244006: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-30 08:13:33.288002: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-30 08:13:34.342324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[2025-04-30 08:13:34,749] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/models/Qwen/Qwen3-8B
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

[INFO:swift] Loading the model using model_dir: /mnt/workspace/qwen3/output/v0-20250430-065954/checkpoint-102
[INFO:swift] Create the default_template for the infer_engine
[INFO:swift] Successfully registered `/usr/local/lib/python3.11/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] Successfully loaded /mnt/workspace/qwen3/output/v0-20250430-065954/checkpoint-102/args.json.
[INFO:swift] default_system: None
[INFO:swift] response_prefix: ''
[INFO:swift] agent_template: hermes
[INFO:swift] max_length: 40960
[INFO:swift] norm_bbox: norm1000


response: <think>
Okay, the user asked, "who are you?" I need to respond appropriately. First, I should introduce myself as an AI assistant developed by swift. I should mention my name, swift-robot, and my purpose to assist with information and tasks. I should keep the response friendly and open-ended, inviting them to ask questions. I should avoid any technical jargon and make sure the tone is approachable. Let me check if there's anything else I need to include. Maybe a brief mention of my capabilities, like answering questions, providing information, and helping with tasks. Also, I should make sure to use proper grammar and punctuation. Alright, that should cover it.
</think>

Hello! I am swift-robot, an artificial intelligence assistant developed by swift. My purpose is to assist you in answering questions, providing information, and helping with various tasks. If you have any questions or need assistance, feel free to ask me!


使用CLI进行 merge lora：

In [ ]:
!CUDA_VISIBLE_DEVICES=0 \
swift export \
    --adapters output/vx-xxx/checkpoint-xxx \
    --merge_lora true

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


run sh: `/usr/local/bin/python /data/joey.wang/llm/ms-swift/swift/cli/export.py --adapters output/vx-xxx/checkpoint-xxx --merge_lora true`
[INFO:swift] Successfully registered `/data/joey.wang/llm/ms-swift/swift/llm/dataset/data/dataset_info.json`.
Traceback (most recent call last):
  File "/data/joey.wang/llm/ms-swift/swift/cli/export.py", line 2, in <module>
    from swift.llm import export_main
  File "<frozen importlib._bootstrap>", line 1075, in _handle_fromlist
  File "/data/joey.wang/llm/ms-swift/swift/utils/import_utils.py", line 94, in __getattr__
    module = self._get_module(self._class_to_module[name])
  File "/data/joey.wang/llm/ms-swift/swift/utils/import_utils.py", line 103, in _get_module
    return importlib.import_module('.' + module_name, self.__name__)
  File "/usr/local/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/data/joey.wang/llm/ms-swift/swift/llm/export/__init__.py", 

### Qwen3-8B GRPO
以Qwen3-8B为例，下面使用swift框架对进行GRPO训练。更多关于GRPO，可以参考GRPO文档：https://swift.readthedocs.io/zh-cn/latest/Instruction/GRPO.html

使用AI-MO/NuminaMath-TIR作为数据集，并使用accuracy函数计算模型回答的准确率奖励, 计算奖励需要安装以下环境：

```bash
pip install math_verify==0.5.2
```
自定义数据集格式与SFT类似，其中assistant部分不必需。如果使用accuracy奖励，则需要solution列来计算准确率。

```jsonl
{"messages": [{"role": "system", "content": "You are a useful and harmless assistant"}, {"role": "user", "content": "Tell me tomorrow's weather"}]}
{"messages": [{"role": "system", "content": "You are a useful and harmless math calculator"}, {"role": "user", "content": "What is 1 + 1?"}, {"role": "assistant", "content": "It equals 2"}, {"role": "user", "content": "What about adding 1?"}]}
{"messages": [{"role": "user", "content": "What is your name?"}]}
```

也可以使用自定义的奖励函数/奖励模型进行训练，数据集中的列会传到奖励函数的`**kwargs`中，自定义奖励函数的例子参考：swift/examples/train/grpo/plugin/plugin.py

```bash
    --external_plugins examples/train/grpo/plugin/plugin.py \
    --reward_funcs external_math_acc external_math_format \
    --reward_model AI-ModelScope/Skywork-Reward-Llama-3.1-8B-v0.2
```
在训练过程中，我们使用vLLM来加速采样过程。设置num_infer_workers=8，我们为每个device都部署一个vLLM engine来加速采样过程。

训练脚本如下：

```bash
# 70G*8
CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 \
NPROC_PER_NODE=8 \
swift rlhf \
    --rlhf_type grpo \
    --model Qwen/Qwen3-8B \
    --train_type full \
    --dataset AI-MO/NuminaMath-TIR \
    --torch_dtype bfloat16 \
    --num_train_epochs 1 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --learning_rate 1e-6 \
    --save_total_limit 2 \
    --logging_steps 5 \
    --output_dir output \
    --gradient_accumulation_steps 1 \
    --warmup_ratio 0.05 \
    --dataloader_num_workers 4 \
    --max_completion_length 4096 \
    --vllm_max_model_len 8192 \
    --reward_funcs accuracy \
    --num_generations 16 \
    --use_vllm true \
    --vllm_gpu_memory_utilization 0.4 \
    --sleep_level 1 \
    --offload_model true \
    --offload_optimizer true \
    --gc_collect_after_offload true \
    --deepspeed zero3 \
    --num_infer_workers 8 \
    --tensor_parallel_size 1 \
    --temperature 1.0 \
    --top_p 0.85 \
    --report_to wandb \
    --log_completions true \
    --overlong_filter true 
```

### Qwen3-30B-A3B MoE SFT（Megatron-SWIFT）

ms-swift引入了Megatron的并行技术来加速大模型的训练，包括数据并行、张量并行、流水线并行、序列并行，上下文并行，专家并行。支持**Qwen3**、**Qwen3-MoE**、Qwen2.5、Llama3、Deepseek-R1蒸馏系等模型的预训练和微调。

对于环境准备（镜像）和HF与MCore模型权重的转换，可以参考Megatron-SWIFT训练文档，这里不进行介绍：https://swift.readthedocs.io/zh-cn/latest/Instruction/Megatron-SWIFT%E8%AE%AD%E7%BB%83.html

我们使用DLC启动训练命令，训练环境是2机8 * 80GiB A800：

更多**多节点启动**方式参考：https://github.com/modelscope/ms-swift/tree/main/examples/train/multi-node

```bash
# https://help.aliyun.com/zh/pai/user-guide/general-environment-variables
# 请确保两个节点的保存权重路径相同
NNODES=$WORLD_SIZE \
NODE_RANK=$RANK \
megatron sft \
    --load Qwen3-30B-A3B-Base-mcore \
    --dataset 'liucong/Chinese-DeepSeek-R1-Distill-data-110k-SFT' \
    --tensor_model_parallel_size 2 \
    --expert_model_parallel_size 8 \
    --moe_grouped_gemm true \
    --moe_shared_expert_overlap true \
    --moe_aux_loss_coeff 0.01 \
    --micro_batch_size 1 \
    --global_batch_size 16 \
    --packing true \
    --recompute_granularity full \
    --recompute_method uniform \
    --recompute_num_layers 1 \
    --train_iters 2000 \
    --eval_iters 50 \
    --finetune true \
    --cross_entropy_loss_fusion true \
    --lr 1e-5 \
    --lr_warmup_iters 100 \
    --min_lr 1e-6 \
    --save megatron_output/Qwen3-30B-A3B-Base \
    --eval_interval 200 \
    --save_interval 200 \
    --max_length 8192 \
    --num_workers 8 \
    --dataset_num_proc 8 \
    --no_save_optim true \
    --no_save_rng true \
    --sequence_parallel true \
    --use_flash_attn true
```
训练loss图（部分）：

<img width="910" alt="Image" src="https://github.com/user-attachments/assets/9fe393aa-8299-4659-aa2f-be5d44f0730b" />

效果截图：

<img width="1066" alt="Image" src="https://github.com/user-attachments/assets/1a924130-1954-43e9-9093-b019aeef5949" />

自定义数据集格式与`swift sft`相同，可以在本文上方找到，指定`--dataset <dataset_path>`即可。

使用`megatron sft`和`swift sft`进行Qwen3-30B-A3B模型全参数训练**速度/显存占用**对比如下：

|          | Megatron-LM | DeepSpeed-ZeRO2 | DeepSpeed-ZeRO3 |
| -------- | ----------- | --------------- | --------------- |
| 训练速度 | 9.6s/it     | -               | 91.2s/it        |
| 显存占用 | 16 * 60GiB  | OOM             | 16 * 80GiB      |


## 模型量化

Qwen3-32B-AWQ: https://modelscope.cn/models/swift/Qwen3-32B-AWQ

Qwen3-30B-A3B-AWQ: https://modelscope.cn/models/swift/Qwen3-30B-A3B-AWQ

## 魔搭大模型训练交流群


<img width="250" alt="Image" src="https://modelscope-open.oss-cn-hangzhou.aliyuncs.com/ms_swift/wechat.png" />


